In [85]:
import mysql.connector
conn = mysql.connector.connect(user='root', password='',
                           host='localhost', database='semtraclus')
print(conn)

In [86]:
from math import radians, cos, sin, asin, sqrt
import os
import glob
from datetime import datetime
import folium
from folium.plugins import MarkerCluster
stay_points = []
def read_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()[6:] # Skip the first 6 lines
        points = []
        for line in lines:
            lat, lon, _, _, _, _, timestamp = line.strip().split(',')
            timestamp = datetime.strptime(timestamp, '%H:%M:%S') # convert timestamp to datetime object
            points.append((float(lat), float(lon),timestamp))

        return points

def manhattan_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def find_stay_points(file_path, threshold_distance, min_duration):
    points = read_file(file_path)
    stay_points = []
    current_stay_point = []
    for i in range(1, len(points)):
        dist = manhattan_distance(points[i][:2], points[i-1][:2])
        if dist <= threshold_distance:
            current_stay_point.append(points[i])
        else:
            if len(current_stay_point) > 0:
                # Check if the current stay point satisfies the minimum duration requirement
                duration = current_stay_point[-1][2] - current_stay_point[0][2]
                if duration.total_seconds() >= min_duration:
                    start_time = current_stay_point[0][2].strftime('%H:%M:%S')
                    end_time = current_stay_point[-1][2].strftime('%H:%M:%S')
                    stay_points.append((current_stay_point[0][:2], current_stay_point[-1][:2], duration.total_seconds(), start_time, end_time))
                current_stay_point = []
    return stay_points


if __name__ == '__main__':
    # Directory containing the .plt files
    directory = 'C:/Users/farha/dataset1/'
    # Threshold distance for defining a stay point
    threshold_distance = 0.005 # in degrees (approx. 555 meters)
    # Minimum duration for a stay point
    min_duration = 300 # in number of points
    
    all_stay_points = []
    # Initialize a folium Map object centered at the first point in the first file
    
    file_path = glob.glob(os.path.join(directory, '*.plt'))[0]
    points = read_file(file_path)
    m = folium.Map(location=[points[0][0], points[0][1]], zoom_start=12)
    
    
    # Loop through all .plt files in the directory and find stay points
    for file_path in glob.glob(os.path.join(directory, '*.plt')):
        stay_points = find_stay_points(file_path, threshold_distance, min_duration)
        for i, point in enumerate(stay_points):
            all_stay_points += (point,)
            #print(f'All stay points: {all_stay_points}')
       # print(f'Stay points in {file_path}:')
        for i, point in enumerate(stay_points):
            print(f'Stay points in {file_path}:')
            print(f'Stay point {i+1}: Latitude={point[0][0]}, Longitude={point[0][1]}, Duration={point[2]} seconds')
            folium.Marker(location=[point[0][0], point[0][1]], popup=f"lat: {point[0][0]}, long: {point[0][1]}").add_to(m)
    # Create a marker cluster
    marker_cluster = MarkerCluster().add_to(m)
    # Display the map
    display(m)


Stay points in C:/Users/farha/dataset1\20081020092505.plt:
Stay point 1: Latitude=39.975217, Longitude=116.329255, Duration=734.0 seconds
Stay points in C:/Users/farha/dataset1\20081102030834.plt:
Stay point 1: Latitude=40.0136, Longitude=116.306569, Duration=4231.0 seconds
Stay points in C:/Users/farha/dataset1\20081102032600.plt:
Stay point 1: Latitude=39.180859, Longitude=117.172596, Duration=10625.0 seconds
Stay points in C:/Users/farha/dataset1\20081102032600.plt:
Stay point 2: Latitude=39.866115, Longitude=116.383602, Duration=2752.0 seconds
Stay points in C:/Users/farha/dataset1\20081104054859.plt:
Stay point 1: Latitude=39.978174, Longitude=116.325782, Duration=24500.0 seconds
Stay points in C:/Users/farha/dataset1\20081112091111.plt:
Stay point 1: Latitude=39.93653, Longitude=116.3480349, Duration=910.0 seconds
Stay points in C:/Users/farha/dataset1\20081120005029.plt:
Stay point 1: Latitude=39.982011, Longitude=116.329123, Duration=37793.0 seconds
Stay points in C:/Users/farh

In [87]:

cursor = conn.cursor()

# create a table for the stay points
staypoints_table_name = 'test3'
create_staypoints_table_query = f"""CREATE TABLE IF NOT EXISTS {staypoints_table_name} (
                        id INT AUTO_INCREMENT PRIMARY KEY,
                        location POINT NOT NULL,
                        duration INT,
                        loc_id VARCHAR(20)
                        );"""
cursor.execute(create_staypoints_table_query)

# create a table for the repeated rows
repeated_rows_table_name = 'rep2'
create_repeated_rows_table_query = f"""CREATE TABLE IF NOT EXISTS {repeated_rows_table_name} (
                        id INT AUTO_INCREMENT PRIMARY KEY,
                        location POINT NOT NULL,
                        duration INT,
                        loc_id VARCHAR(20),
                        loc_ref VARCHAR(20)
                        );"""
cursor.execute(create_repeated_rows_table_query)

# insert the stay points into the staypoints table
insert_query = f"""INSERT INTO {staypoints_table_name} (location, duration, loc_id) 
                    VALUES (ST_PointFromText(%s), %s, %s)"""

for i, point in enumerate(all_stay_points):
    loc_id = f"{point[0][0]:.6f}{point[0][1]:.6f}"
    data = (f'POINT({point[0][1]} {point[0][0]})', point[2], loc_id)
    cursor.execute(insert_query, data)

# retrieve the stay points and print the results
cursor.execute("SELECT id,ST_AsText(location),duration,loc_id FROM test3")
staypoint_rows = cursor.fetchall()
for row in staypoint_rows:
    print(row)

# iterate through the staypoint_rows and check for points within 10 meters
for i, row1 in enumerate(staypoint_rows):
    for j, row2 in enumerate(staypoint_rows[i+1:]):
        # get the latitude and longitude coordinates of the two points
        lat1, lon1 = row1[1].replace("POINT(", "").replace(")", "").split(" ")
        lat2, lon2 = row2[1].replace("POINT(", "").replace(")", "").split(" ")
        #print(lat1,lon1)
        #print(lat2,lon2)
        # calculate the distance between the two points in meters
        distance = 6371000 * 2 * asin(sqrt(sin(radians(float(lat2)-float(lat1))/2)**2 + cos(radians(float(lat1))) * cos(radians(float(lat2))) * sin(radians(float(lon2)-float(lon1))/2)**2))
        if distance < 10:  # if distance between two points is less than 10 meters
            # insert the repeated row into the repeated_rows table
            insert_repeated_row_query = f"""INSERT INTO {repeated_rows_table_name} (location, duration, loc_id,loc_ref) 
                                            VALUES (ST_PointFromText(%s), %s, %s,%s)"""
            cursor.execute(insert_repeated_row_query, (row2[1], row2[2], row2[3],row1[3]))

# retrieve and print the results from the repeated_rows table
cursor.execute(f"SELECT id,ST_AsText(location),duration,loc_id,loc_ref FROM {repeated_rows_table_name}")
repeated_rows = cursor.fetchall()
print("Repeated rows:")
for row in repeated_rows:
    print(row)

# commit the changes and close the connection
conn.commit()
conn.close()


(1, 'POINT(116.329255 39.975217)', 734, '39.975217116.329255')
(2, 'POINT(116.306569 40.0136)', 4231, '40.013600116.306569')
(3, 'POINT(117.172596 39.180859)', 10625, '39.180859117.172596')
(4, 'POINT(116.383602 39.866115)', 2752, '39.866115116.383602')
(5, 'POINT(116.325782 39.978174)', 24500, '39.978174116.325782')
(6, 'POINT(116.3480349 39.93653)', 910, '39.936530116.348035')
(7, 'POINT(116.329123 39.982011)', 37793, '39.982011116.329123')
(8, 'POINT(116.312201 40.008211)', 16183, '40.008211116.312201')
(9, 'POINT(116.801085 39.949695)', 1095, '39.949695116.801085')
(10, 'POINT(116.782622 39.946412)', 16381, '39.946412116.782622')
(11, 'POINT(116.319737 40.007648)', 2327, '40.007648116.319737')
(12, 'POINT(116.457307 39.948709)', 10463, '39.948709116.457307')
(13, 'POINT(116.430476 39.907678)', 1440, '39.907678116.430476')
(14, 'POINT(116.319697 40.007705)', 978, '40.007705116.319697')
(15, 'POINT(116.31287 39.983983)', 23577, '39.983983116.312870')
(16, 'POINT(116.319571 40.007591)

In [89]:
import folium

# create a map centered on the first stay point
#map_center = staypoint_rows[0][1].replace("POINT(", "").replace(")", "").split(" ")
#m = folium.Map(location=[float(map_center[0]), float(map_center[1])], zoom_start=15)

# get the latitude and longitude of the first stay point
lat_lon = staypoint_rows[0][1].replace("POINT(", "").replace(")", "").split(" ")

# create a map centered on the first stay point
m = folium.Map(location=[float(lat_lon[0]), float(lat_lon[1])], zoom_start=15)


# add markers for the stay points
for row in staypoint_rows:
    lat_lon = row[1].replace("POINT(", "").replace(")", "").split(" ")
    folium.Marker(location=[float(lat_lon[0]), float(lat_lon[1])], popup=f"Duration: {row[2]}").add_to(m)

# add markers for the repeated rows with a different color
for row in repeated_rows:
    lat_lon = row[1].replace("POINT(", "").replace(")", "").split(" ")
    folium.Marker(location=[float(lat_lon[0]), float(lat_lon[1])], popup=f"Duration: {row[2]}, Loc Ref: {row[4]}", icon=folium.Icon(color='red')).add_to(m)

# show the map
m


In [94]:
import folium

# Create a map centered on the first stay point
map_center = staypoint_rows[0][1].replace("POINT(", "").replace(")", "").split(" ")
m = folium.Map(location=[float(map_center[0]), float(map_center[1])], zoom_start=15, tiles='OpenStreetMap')

# Add markers for the stay points
for row in staypoint_rows:
    loc = row[1].replace("POINT(", "").replace(")", "").split(" ")
    folium.Marker(location=[float(loc[0]), float(loc[1])], icon=folium.Icon(color='blue')).add_to(m)

# Add markers for the revisited points
for row in repeated_rows:
    loc = row[1].replace("POINT(", "").replace(")", "").split(" ")
    folium.Marker(location=[float(loc[0]), float(loc[1])], icon=folium.Icon(color='red')).add_to(m)

m
